<a href="https://colab.research.google.com/github/Giovanni-Chipuli/Churn_Clasif_Despliegue/blob/main/Churn_3_Despliegue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Listado de Dependencias

In [23]:
import numpy as pd
import pandas as pd
import matplotlib.pyplot as plt
import requests
import pickle as pkl
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

#Seteo de Parámetros

In [24]:
periodo = 202301       #fecha de dataset con datos de nuevos clientes (enero del 2023, por ejemplo)

#nombre del nuevo dataset (para este cuaderno, la dirección de descarga de un dataset que simula nuevos datos)
FileNewData = 'https://raw.githubusercontent.com/big-gio/02.Clasif_Despliegue-Bank_Churn/main/Archivos/Churn_Modelling_New.csv'
#nombre del archivo que contiene el modelo seleccionado
FileModel = 'https://raw.githubusercontent.com/big-gio/02.Clasif_Despliegue-Bank_Churn/main/Archivos/Modelo_Churn_Final.pkl'
FileDataDelivery = f'df_Modelo_Churn_{periodo}.gzip'
id = 'CustomerId'
target = 'Exited'

#Data Preparation

In [25]:
cm = pd.read_csv(FileNewData)                        #el archivo con nuevos datos se guarda en un objeto

#se carga el archivo .pkl que contiene el modelo seleccionado y se guarda en un objeto al que se le llama "modelo"
response = requests.get(FileModel)
with open('Modelo_Churn_Final.pkl', 'wb') as f:
    f.write(response.content)
modelo = pkl.load(open('Modelo_Churn_Final.pkl', 'rb'))

#tabla única
var_predict = list(set(cm.columns)-{'Surname'} - {id})
cmTU = cm[[id]+var_predict]

#Tratamiento de los datos categóricos
le = LabelEncoder()
le.fit(cmTU['Gender'])
cmTU['Gender'] = le.transform(cmTU['Gender'])
le.fit(cmTU['Geography'])
cmTU['Geography'] = le.transform(cmTU['Geography'])

#tratamiento de datos. Normalización
cmTU_esc = cmTU[['Age', 'EstimatedSalary', 'Balance', 'CreditScore']].copy()

scaler = StandardScaler()
scaler.fit(cmTU_esc)
cmTU_esc2 = scaler.transform(cmTU_esc)

cmTU_esc2 = pd.DataFrame(cmTU_esc2)
cmTU_esc2.columns= ['Age_esc', 'EstimatedSalary_Esc', 'Balance_esc', 'CreditScore_esc']
cmTU2 = pd.concat([cmTU, cmTU_esc2], axis= 1)
cmTU2.drop(['Age', 'EstimatedSalary', 'Balance', 'CreditScore'], axis = 'columns', inplace= True)

#tratamiento de datos. Atípicos
var_out = ['Age_esc', 'CreditScore_esc', 'NumOfProducts']
for col in var_out:
  p25= cmTU2[col].describe()['25%']
  p75= cmTU2[col].describe()['75%']
  IQR= p75 - p25
  minbox = p25-(1.5*IQR)
  maxbox = p75+(1.5*IQR)
  cmTU2[col] = cmTU2[col].apply(lambda x: minbox if x<minbox else x)
  cmTU2[col] = cmTU2[col].apply(lambda x: maxbox if x>maxbox else x)

#se obtienen las variables finales generadas en el cuaderno anterior
var_predict = list(modelo.feature_names_in_)

#separación de datos
X = cmTU2[var_predict]
#ya no se crea el objeto "y" que contenga a "Target", ya que es lo que se pretende clasificar

<ipython-input-25-60306bd2e88d>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cmTU['Gender'] = le.transform(cmTU['Gender'])
<ipython-input-25-60306bd2e88d>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cmTU['Geography'] = le.transform(cmTU['Geography'])


#Inferencia

In [26]:
cmTU2[f'{target}_pred'] = modelo.predict(X)
cmTU2['Prob_pred'] = modelo.predict_proba(X)[:,1]

#Delivery

In [27]:
cmTU2.to_parquet(FileDataDelivery)

In [28]:
#predicciones
cmTU2.head()

,CustomerId,Geography,Tenure,IsActiveMember,HasCrCard,Gender,NumOfProducts,Age_esc,EstimatedSalary_Esc,Balance_esc,CreditScore_esc,Exited_pred,Prob_pred
0,15600882,2,7,1,1,0,2.0,-0.399717,-0.629019,-1.162719,-0.099752,0,0.014888
1,15643966,1,3,1,0,1,2.0,0.629742,-0.657489,1.106816,-0.303227,0,0.183761
2,15597945,2,8,0,1,0,2.0,-0.708555,0.643480,-1.162719,-0.089042,0,0.014888
3,15750181,1,9,0,0,1,2.0,0.217958,-0.349520,0.583283,-0.977908,0,0.092827
4,15789484,1,6,1,1,0,2.0,-0.296771,-1.298418,1.530217,1.142519,0,0.092827
